Disclosure: this is taken from  https://kaggle.com/zaharch/keras-model-boosted-with-plates-leak

As reported by Recursion [in this post](https://www.kaggle.com/c/recursion-cellular-image-classification/discussion/102905), there is a special structure in the data which simplifies predictions significantly.

Assignments of sirnas to plates is not completely random in this competition. In this kernel, first I show it on the train data, and then apply the leak [on the pretrained Keras model](https://www.kaggle.com/chandyalex/recursion-cellular-keras-densenet) (kudos to [Alex](https://www.kaggle.com/chandyalex)) with LB 0.113 to get score 0.207. Same model which uses 2 sites for inference gets LB score 0.231 (the original model uses only one site but I just can't hold myself on that). 

In [1]:
!pip install -q -r requirements.txt

menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 3.0.2 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.4.1 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import os

from tqdm import tqdm
import PIL
import cv2
from PIL import Image, ImageOps

from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.densenet import DenseNet121
import keras
from keras.models import Model

# me
import torch
import torch.nn as nn
import torch.utils.data as D
from torch.optim.lr_scheduler import ExponentialLR
import torch.nn.functional as F
from torch.autograd import Variable

from torchvision import transforms

from ignite.engine import Events
from scripts.ignite import create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers.tqdm_logger import ProgressBar
from ignite.handlers import  EarlyStopping, ModelCheckpoint
from ignite.contrib.handlers import LinearCyclicalScheduler, CosineAnnealingScheduler

from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split

from efficientnet_pytorch import EfficientNet

from scripts.evaluate import eval_model 

SIZE = 224
NUM_CLASSES = 1108

train_csv = pd.read_csv("../input/rxrxaicsv/train.csv")
test_csv = pd.read_csv("../input/rxrxaicsv/test.csv")
sub = pd.read_csv("./submissiont.csv")

Using TensorFlow backend.


# Train data

In [2]:
sub.head()

id_code  sirna
0  HEPG2-08_1_B03    150
1  HEPG2-08_1_B04   1088
2  HEPG2-08_1_B05    614
3  HEPG2-08_1_B06    405
4  HEPG2-08_1_B07    189

Look at the first 10 sirnas plates assignments across the train set. One can observe that two sirnas that are on the same plate in the first experiment stay on the same plate for all experiments. Moreover, there are only 3 unique rows. 

In [3]:
np.stack([train_csv.plate.values[train_csv.sirna == i] for i in range(10)]).transpose()

array([[4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [2, 3, 4, 3, 1, 3, 3, 4, 3, 2],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [2, 3, 4, 3, 1, 3, 3, 4, 3, 2],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [3, 4, 1, 4, 2, 4, 4, 1, 4, 3],
       [3, 4, 1, 4, 2, 4, 4, 1, 4, 3],
       [2, 3, 4, 3, 1, 3, 3, 4, 3, 2],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [2, 3, 4, 3, 1, 3, 3, 4, 3, 2],
       [2, 3, 4, 3, 1, 3, 3, 4, 3, 2],
       [3, 4, 1, 4, 2, 4, 4, 1, 4, 3],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [4, 1, 2, 1, 3, 1, 1, 2, 1, 4],
       [3, 4, 1, 4, 2, 4,

The above observation can be easily verified on the whole train data, - there are 4 groups of 277 sirnas in each group which stick together. 

But is there any order of groups to plates assignment? In general, there are `4*3*2=24` possible combinations of assigning 4 groups to 4 plates. But in the train data only 3 are active, each assignment appearing 22, 7 and 4 times respectively.

In [4]:
# you will see the same output here for each sirna number
train_csv.loc[train_csv.sirna==0,'plate'].value_counts()

4    22
2     7
3     4
Name: plate, dtype: int64

Later we will see that the 4th combination, missing from the training data, does in fact appear in the test data. My conlusion here is that Recursion used some kind of rotation of plates only, therefore only 4 combinations.

Let's calculate which sirna belongs to which plate in every of the 4 assignments:

In [5]:
plate_groups = np.zeros((1108,4), int)
for sirna in range(1108):
    grp = train_csv.loc[train_csv.sirna==sirna,:].plate.value_counts().index.values
    assert len(grp) == 3
    plate_groups[sirna,0:3] = grp
    plate_groups[sirna,3] = 10 - grp.sum()
    
plate_groups[:10,:]

array([[4, 2, 3, 1],
       [1, 3, 4, 2],
       [2, 4, 1, 3],
       [1, 3, 4, 2],
       [3, 1, 2, 4],
       [1, 3, 4, 2],
       [1, 3, 4, 2],
       [2, 4, 1, 3],
       [1, 3, 4, 2],
       [4, 2, 3, 1]])

# Test data

Now let's take a look if we observe the same behavior in the test data. I use the output predictions from the kernel that I mentioned to calculate average probability of each assignment for every experiment.

In [6]:
all_test_exp = test_csv.experiment.unique()

group_plate_probs = np.zeros((len(all_test_exp),4))
for idx in range(len(all_test_exp)):
    preds = sub.loc[test_csv.experiment == all_test_exp[idx],'sirna'].values
    pp_mult = np.zeros((len(preds),1108))
    pp_mult[range(len(preds)),preds] = 1
    
    sub_test = test_csv.loc[test_csv.experiment == all_test_exp[idx],:]
    assert len(pp_mult) == len(sub_test)
    
    for j in range(4):
        mask = np.repeat(plate_groups[np.newaxis, :, j], len(pp_mult), axis=0) == \
               np.repeat(sub_test.plate.values[:, np.newaxis], 1108, axis=1)
        
        group_plate_probs[idx,j] = np.array(pp_mult)[mask].sum()/len(pp_mult)

In [7]:
pd.DataFrame(group_plate_probs, index = all_test_exp)

0         1         2         3
HEPG2-08  0.166215  0.166215  0.164408  0.503162
HEPG2-09  0.198556  0.402527  0.237365  0.161552
HEPG2-10  0.583032  0.140794  0.139892  0.136282
HEPG2-11  0.615732  0.123870  0.118445  0.141953
HUVEC-17  0.664260  0.101083  0.112816  0.121841
HUVEC-18  0.570009  0.142728  0.142728  0.144535
HUVEC-19  0.072202  0.086643  0.783394  0.057762
HUVEC-20  0.046029  0.038809  0.845668  0.069495
HUVEC-21  0.121841  0.144404  0.129061  0.604693
HUVEC-22  0.712996  0.092960  0.102888  0.091155
HUVEC-23  0.658781  0.111920  0.111920  0.117379
HUVEC-24  0.099091  0.100000  0.100000  0.700909
RPE-08    0.148917  0.490072  0.168773  0.192238
RPE-09    0.603433  0.146341  0.119241  0.130985
RPE-10    0.650722  0.118231  0.111011  0.120036
RPE-11    0.596892  0.124314  0.147166  0.131627
U2OS-04   0.258123  0.254513  0.235560  0.251805
U2OS-05   0.187785  0.206928  0.206016  0.399271

Let's select the most probable assignment for every test experiment. You may say that some of the selections here are not certain and the probabilities are too close. But we get the same assignments with our much better models, so even this relatively simple model is able to make correct assignments.

In [25]:
exp_to_group1 = group_plate_probs.argmax(1)
exp_to_group = exp_to_group1.copy()
exp_to_group[-2] = 2

# Running predictions with the existing efficientnet-3 model

In the code below we load the model, make predictions to get the full probabilites matrix, and set 3 out of 4 plates for every sirna to zero, according to the assignment that we previously selected.

In [9]:
img_dir = '../input/rxrxairgb512'
path_data = '../input/rxrxaicsv'
device = 'cuda'
batch_size = 8
torch.manual_seed(0)
model_name = 'efficientnet-b3'

In [10]:
jitter = (0.6, 1.4)
class ImagesDS(D.Dataset):
    # taken textbook from https://arxiv.org/pdf/1812.01187.pdf
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(448),
        transforms.ColorJitter(brightness=jitter, contrast=jitter, saturation=jitter, hue=.1),
        transforms.RandomHorizontalFlip(p=0.5),
        # PCA Noise should go here,
        transforms.ToTensor(),
        transforms.Normalize(mean=(123.68, 116.779, 103.939), std=(58.393, 57.12, 57.375))
    ])
    
    transform_validation = transforms.Compose([
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize(mean=(123.68, 116.779, 103.939), std=(58.393, 57.12, 57.375))
    ])

    def __init__(self, df, img_dir=img_dir, mode='train', validation=False, site=1):
        self.records = df.to_records(index=False)
        self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]
        self.validation = validation
        
    @staticmethod
    def _load_img_as_tensor(file_name, validation):
        with Image.open(file_name) as img:
            if not validation:
                return ImagesDS.transform_train(img)
            else:
                return ImagesDS.transform_validation(img)

    def _get_img_path(self, index, site=1):
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return f'{self.img_dir}/{self.mode}/{experiment}_{plate}_{well}_s{site}.jpeg'
        
    def __getitem__(self, index):
        img1, img2 = [self._load_img_as_tensor(self._get_img_path(index, site), self.validation) for site in [1,2]]
        if self.mode == 'train':
            return img1, img2, int(self.records[index].sirna)
        else:
            return img1, img2, self.records[index].id_code

    def __len__(self):
        return self.len

In [11]:
# dataframes for training, cross-validation, and testing
df_test = pd.read_csv(path_data+'/test.csv')
ds_test = ImagesDS(df_test, mode='test', validation=True)
tloader = D.DataLoader(ds_test, batch_size=1, shuffle=False, num_workers=4)

In [12]:
class EfficientNetTwoInputs(nn.Module):
    def __init__(self):
        super(EfficientNetTwoInputs, self).__init__()
        self.classes = 1108
        
        model = model = EfficientNet.from_pretrained(model_name, num_classes=1108) 
        num_ftrs = model._fc.in_features
        model._fc = nn.Identity()
        
        self.resnet = model
        self.fc = nn.Linear(num_ftrs * 2, self.classes)

    def forward(self, x1, x2):
        x1_out = self.resnet(x1)
        x2_out = self.resnet(x2)
   
        N, _, _, _ = x1.size()
        x1_out = x1_out.view(N, -1)
        x2_out = x2_out.view(N, -1)
        
        out = torch.cat((x1_out, x2_out), 1)
        out = self.fc(out)

        return out 
    
model = EfficientNetTwoInputs()
model.load_state_dict(torch.load('models/Model_efficientnet-b3_93.pth'))
model.eval()
model.cuda()

Loaded pretrained weights for efficientnet-b3


EfficientNetTwoInputs(
  (resnet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )

In [13]:
with torch.no_grad():
    predicted = []
    for x1, x2, _ in tqdm_notebook(tloader): 
        x1 = x1.to(device)
        x2 = x2.to(device)
        output = model(x1,x2)
        idx = output.cpu().numpy()
        predicted.append(idx[0])

In [14]:
predicted = np.stack(predicted).squeeze()

this is the function that sets 75% of the sirnas to zero according to the selected assignment:

In [26]:
def select_plate_group(pp_mult, idx):
    sub_test = test_csv.loc[test_csv.experiment == all_test_exp[idx],:]
    assert len(pp_mult) == len(sub_test)
    mask = np.repeat(plate_groups[np.newaxis, :, exp_to_group[idx]], len(pp_mult), axis=0) != \
           np.repeat(sub_test.plate.values[:, np.newaxis], 1108, axis=1)
    pp_mult[mask] = -1000000.
    return pp_mult

In [27]:
for idx in range(len(all_test_exp)):
    #print('Experiment', idx)
    indices = (test_csv.experiment == all_test_exp[idx])

    preds = predicted[indices,:].copy()
    
    preds = select_plate_group(preds, idx)
    sub.loc[indices,'sirna'] = preds.argmax(1)

In [28]:
(sub.sirna == pd.read_csv("./submissiont.csv").sirna).mean()

0.5900889581343921

In [29]:
sub.to_csv('./working/submission3.csv', index=False, columns=['id_code','sirna'])

That is all! I hope this demonstration will get everyone to the same playing field on this issue. Assigning 277 sirnas to 277 wells is still a hard problem to crack.